In [1]:
import csv
import plotly.graph_objects as go

In [2]:
# File paths (adjust as needed)
routes_file = "routes.dat"
airports_file = "airports.dat"

# Data structures
nodes = {}
edges = []

In [3]:
# Step 1: Read airports.dat and create nodes
with open(airports_file, "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    for row in reader:
        # Assuming the column order matches the format
        airport_id = row[0]
        airport_name = row[1]
        city = row[2]
        country = row[3]
        latitude = float(row[6])
        longitude = float(row[7])
        
        # Add the airport node to the dictionary
        nodes[airport_id] = {
            "name": airport_name,
            "city": city,
            "country": country,
            "latitude": latitude,
            "longitude": longitude
        }

In [4]:
# Step 2: Read routes.dat and create edges
with open(routes_file, "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    for row in reader:
        # Assuming the column order matches the format
        airline = row[0]
        source_id = row[3]
        destination_id = row[5]
        
        # Skip edges where source or destination airport IDs are missing
        if source_id in nodes and destination_id in nodes:
            edges.append({
                "airline": airline,
                "source": nodes[source_id],
                "destination": nodes[destination_id]
            })

In [5]:
# Output the results
print("Nodes (Airports):")
for airport_id, attributes in nodes.items():
    print(f"{airport_id}: {attributes}")
    break

print("\nEdges (Routes):")
for edge in edges:
    print(edge)
    break

Nodes (Airports):
1: {'name': 'Goroka Airport', 'city': 'Goroka', 'country': 'Papua New Guinea', 'latitude': -6.081689834590001, 'longitude': 145.391998291}

Edges (Routes):
{'airline': '2B', 'source': {'name': 'Sochi International Airport', 'city': 'Sochi', 'country': 'Russia', 'latitude': 43.449902, 'longitude': 39.9566}, 'destination': {'name': 'Kazan International Airport', 'city': 'Kazan', 'country': 'Russia', 'latitude': 55.606201171875, 'longitude': 49.278701782227}}


In [11]:
import networkx as nx

# Create an empty graph
G = nx.Graph()

# Add nodes (airports) to the graph
for airport_id, attributes in nodes.items():
    G.add_node(airport_id, **attributes)

# Add edges (routes) to the graph
for edge in edges:
    source = edge['source']['name']
    destination = edge['destination']['name']
    G.add_edge(source, destination, airline=edge['airline'])

# Compute degree centrality
degree_centrality = nx.degree_centrality(G)

# Sort by degree centrality
sorted_degree = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)

# Print top 10 hubs
for airport_name, centrality in sorted_degree[:10]:
    print(f"Airport: {airport_name}, Degree] Centrality: {centrality}")

Airport: Amsterdam Airport Schiphol, Degree] Centrality: 0.022756469076894843
Airport: Frankfurt am Main Airport, Degree] Centrality: 0.022389429253073957
Airport: Charles de Gaulle International Airport, Degree] Centrality: 0.022022389429253074
Airport: Atatürk International Airport, Degree] Centrality: 0.021380069737566527
Airport: Hartsfield Jackson Atlanta International Airport, Degree] Centrality: 0.019911910442282986
Airport: Chicago O'Hare International Airport, Degree] Centrality: 0.018902550926775553
Airport: Beijing Capital International Airport, Degree] Centrality: 0.018810790970820335
Airport: Munich Airport, Degree] Centrality: 0.01761791154340246
Airport: Domodedovo International Airport, Degree] Centrality: 0.017250871719581572
Airport: Dallas Fort Worth International Airport, Degree] Centrality: 0.017250871719581572


In [15]:
import plotly.express as px
import pandas as pd

# Extract airport data for top 10
top_airports = []
for airport_name, centrality in sorted_degree[:10]:
    for edge in edges:
        if edge['source']['name'] == airport_name:
            top_airports.append({
                'name': airport_name,
                'city': edge['source']['city'],
                'country': edge['source']['country'],
                'latitude': edge['source']['latitude'],
                'longitude': edge['source']['longitude'],
                'centrality': centrality
            })
            break

# Convert to DataFrame
df_top_airports = pd.DataFrame(top_airports)

# Plotting the map
fig = px.scatter_geo(
    df_top_airports,
    lat='latitude',
    lon='longitude',
    #text='name',
    hover_name='city',
    size='centrality',
    projection='natural earth',
    title='Top 10 Airports by Centrality'
)

fig.update_layout(geo=dict(showland=True, landcolor="lightgray"))

fig.show()

In [17]:
# Compute betweenness centrality
betweenness_centrality = nx.betweenness_centrality(G)

# Sort by betweenness centrality
sorted_betweenness = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)

# Print top 10 airports by betweenness centrality
for airport_name, centrality in sorted_betweenness[:10]:
    print(f"Airport: {airport_name}, Betweenness Centrality: {centrality}")

Airport: Charles de Gaulle International Airport, Betweenness Centrality: 0.005420207740343245
Airport: Los Angeles International Airport, Betweenness Centrality: 0.005135671671109709
Airport: Ted Stevens Anchorage International Airport, Betweenness Centrality: 0.005125209788449216
Airport: Dubai International Airport, Betweenness Centrality: 0.004628412516093334
Airport: Frankfurt am Main Airport, Betweenness Centrality: 0.004504988422254272
Airport: Amsterdam Airport Schiphol, Betweenness Centrality: 0.004374268350642617
Airport: Beijing Capital International Airport, Betweenness Centrality: 0.0042865324730917585
Airport: Chicago O'Hare International Airport, Betweenness Centrality: 0.0040040633219087865
Airport: Lester B. Pearson International Airport, Betweenness Centrality: 0.0038891168400845863
Airport: Atatürk International Airport, Betweenness Centrality: 0.0036674263096069092


In [18]:
# Extract airport data for top 10
top_airports = []
for airport_name, centrality in sorted_betweenness[:10]:
    for edge in edges:
        if edge['source']['name'] == airport_name:
            top_airports.append({
                'name': airport_name,
                'city': edge['source']['city'],
                'country': edge['source']['country'],
                'latitude': edge['source']['latitude'],
                'longitude': edge['source']['longitude'],
                'centrality': centrality
            })
            break

# Convert to DataFrame
df_top_airports = pd.DataFrame(top_airports)

# Plotting the map
fig = px.scatter_geo(
    df_top_airports,
    lat='latitude',
    lon='longitude',
    #text='name',
    hover_name='city',
    size='centrality',
    projection='natural earth',
    title='Top 10 Airports by Betweenness Centrality'
)

fig.update_layout(geo=dict(showland=True, landcolor="lightgray"))

fig.show()

In [19]:
# Compute eigenvector centrality
eigenvector_centrality = nx.eigenvector_centrality(G)

# Sort by eigenvector centrality
sorted_eigenvector = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)

# Print top 10 airports by eigenvector centrality
for airport_name, centrality in sorted_eigenvector[:10]:
    print(f"Airport: {airport_name}, Eigenvector Centrality: {centrality}")

Airport: Amsterdam Airport Schiphol, Eigenvector Centrality: 0.16787862313775015
Airport: Frankfurt am Main Airport, Eigenvector Centrality: 0.16639360556512212
Airport: Charles de Gaulle International Airport, Eigenvector Centrality: 0.15946867299421438
Airport: Munich Airport, Eigenvector Centrality: 0.14850125294474087
Airport: Leonardo da Vinci–Fiumicino Airport, Eigenvector Centrality: 0.13632593305180626
Airport: London Heathrow Airport, Eigenvector Centrality: 0.13597213697411975
Airport: Atatürk International Airport, Eigenvector Centrality: 0.12912841296227298
Airport: Barcelona International Airport, Eigenvector Centrality: 0.12809384878220356
Airport: Zürich Airport, Eigenvector Centrality: 0.12599349543893415
Airport: Adolfo Suárez Madrid–Barajas Airport, Eigenvector Centrality: 0.12296163375221078


In [20]:
# Extract airport data for top 10
top_airports = []
for airport_name, centrality in sorted_eigenvector[:10]:
    for edge in edges:
        if edge['source']['name'] == airport_name:
            top_airports.append({
                'name': airport_name,
                'city': edge['source']['city'],
                'country': edge['source']['country'],
                'latitude': edge['source']['latitude'],
                'longitude': edge['source']['longitude'],
                'centrality': centrality
            })
            break

# Convert to DataFrame
df_top_airports = pd.DataFrame(top_airports)

# Plotting the map
fig = px.scatter_geo(
    df_top_airports,
    lat='latitude',
    lon='longitude',
    #text='name',
    hover_name='city',
    size='centrality',
    projection='natural earth',
    title='Top 10 Airports by sorted_eigenvector'
)

fig.update_layout(geo=dict(showland=True, landcolor="lightgray"))

fig.show()